## Create XML for Object Dection

In [57]:
import glob
import xml.etree.ElementTree as ET

In [58]:
# Define constants
FOLDER = 'images'
#PATH = '/Users/jordanwalker/Code/reference-architecture/python/electric-gauge/psi_gauge/{}'
PATH = '/home/ms/Downloads/OCI/DeepGaugeReader_ProjectFiles/ObjectDetection_RetrainingScript/gauge_dataset/{}'
LOCAL_PATH = "./annotations/{0}.xml"
WIDTH = '224'
HEIGHT = '224'
DEPTH = '1'
XMIN = '10'
YMIN = '10'
XMAX = '214'
YMAX = '214'

In [59]:
def make_xml_file(filename, name, abs_path):
    # define the variables
    FILENAME = filename
    NAME = name
    # create the file structure
    annotation = ET.Element('annotation')
    folder = ET.SubElement(annotation, 'folder')  
    filename = ET.SubElement(annotation, 'filename')
    path = ET.SubElement(annotation, 'path')
    source = ET.SubElement(annotation, 'source')
    database = ET.SubElement(source, 'database')
    size = ET.SubElement(annotation, 'size')
    width = ET.SubElement(size, 'width')
    height = ET.SubElement(size, 'height')
    depth = ET.SubElement(size, 'depth')
    segmented = ET.SubElement(annotation, 'segmented')
    obj = ET.SubElement(annotation, 'object')
    name = ET.SubElement(obj, 'name')
    pose = ET.SubElement(obj, 'pose')
    truncated = ET.SubElement(obj, 'truncated')
    difficult = ET.SubElement(obj, 'difficult')
    bndbox = ET.SubElement(obj, 'bndbox')
    xmin = ET.SubElement(bndbox, 'xmin')
    ymin = ET.SubElement(bndbox, 'ymin')
    xmax = ET.SubElement(bndbox, 'xmax')
    ymax = ET.SubElement(bndbox, 'ymax')

    # Populate the text between the tags
    annotation.set('verified','yes') 
    folder.text = FOLDER
    filename.text = FILENAME
    path.text = PATH.format(abs_path)
    database.text = 'Unknown'
    width.text = WIDTH
    height.text = HEIGHT
    depth.text = DEPTH
    segmented.text = '0'
    name.text = 'gauge'
    pose.text = 'Unspecified'
    truncated.text = '0'
    difficult.text = '0'
    xmin.text = XMIN
    ymin.text = YMIN
    xmax.text = XMAX
    ymax.text = YMAX

    # create a new XML file with the results
    mydata = ET.tostring(annotation)  
    file = LOCAL_PATH.format(NAME)
    myfile = open(file, "wb")  
    myfile.write(mydata) 

In [60]:
dirList = glob.glob("images/*.jpg")
X_img_paths = ['{}'.format(file) for file in dirList]
print (X_img_paths)

['images/gauge-76.jpg', 'images/gauge-110.jpg', 'images/gauge-134.jpg', 'images/gauge-216.jpg', 'images/gauge-124.jpg', 'images/gauge-193.jpg', 'images/gauge-235.jpg', 'images/gauge-91.jpg', 'images/gauge-53.jpg', 'images/gauge-182.jpg', 'images/gauge-29.jpg', 'images/gauge-212.jpg', 'images/gauge-122.jpg', 'images/gauge-74.jpg', 'images/gauge-62.jpg', 'images/gauge-127.jpg', 'images/gauge-115.jpg', 'images/gauge-172.jpg', 'images/gauge-165.jpg', 'images/gauge-55.jpg', 'images/gauge-22.jpg', 'images/gauge-11.jpg', 'images/gauge-209.jpg', 'images/gauge-113.jpg', 'images/gauge-46.jpg', 'images/gauge-38.jpg', 'images/gauge-154.jpg', 'images/gauge-227.jpg', 'images/gauge-233.jpg', 'images/gauge-244.jpg', 'images/gauge-81.jpg', 'images/gauge-157.jpg', 'images/gauge-181.jpg', 'images/gauge-65.jpg', 'images/gauge-27.jpg', 'images/gauge-164.jpg', 'images/gauge-108.jpg', 'images/gauge-143.jpg', 'images/gauge-82.jpg', 'images/gauge-4.jpg', 'images/gauge-239.jpg', 'images/gauge-200.jpg', 'images/

In [61]:
print(len(X_img_paths))
# iterate through the images
for img in X_img_paths:
    # split on / and join the names with an _
    split = img.split('/')
    name = "_".join(split[1:]).replace('.jpg','')
    print(name)
    make_xml_file(split[-1], name, img) 
      

245
gauge-76
gauge-110
gauge-134
gauge-216
gauge-124
gauge-193
gauge-235
gauge-91
gauge-53
gauge-182
gauge-29
gauge-212
gauge-122
gauge-74
gauge-62
gauge-127
gauge-115
gauge-172
gauge-165
gauge-55
gauge-22
gauge-11
gauge-209
gauge-113
gauge-46
gauge-38
gauge-154
gauge-227
gauge-233
gauge-244
gauge-81
gauge-157
gauge-181
gauge-65
gauge-27
gauge-164
gauge-108
gauge-143
gauge-82
gauge-4
gauge-239
gauge-200
gauge-177
gauge-183
gauge-141
gauge-112
gauge-66
gauge-130
gauge-83
gauge-89
gauge-217
gauge-97
gauge-78
gauge-71
gauge-116
gauge-37
gauge-135
gauge-43
gauge-245
gauge-150
gauge-138
gauge-171
gauge-156
gauge-238
gauge-120
gauge-220
gauge-54
gauge-80
gauge-79
gauge-10
gauge-107
gauge-149
gauge-125
gauge-204
gauge-224
gauge-33
gauge-114
gauge-52
gauge-123
gauge-41
gauge-206
gauge-69
gauge-144
gauge-92
gauge-145
gauge-139
gauge-102
gauge-210
gauge-166
gauge-186
gauge-103
gauge-60
gauge-47
gauge-106
gauge-21
gauge-221
gauge-26
gauge-100
gauge-215
gauge-117
gauge-162
gauge-96
gauge-167
gauge

 ## Convert xml files ot csv

In [62]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    image_path = os.path.join(os.getcwd(), 'annotations')
    xml_df = xml_to_csv(image_path)
    xml_df.to_csv('./data/gauge_labels.csv', index=None)
    print('Successfully converted xml to csv.')


main()

Successfully converted xml to csv.


 ## Split the csv file into train and test

In [63]:
import numpy as np
import pandas as pd
np.random.seed(1)


In [64]:
full_labels = pd.read_csv('./data/gauge_labels.csv')
full_labels.head()
len(full_labels)


245

In [65]:
grouped = full_labels.groupby('filename')
len(grouped)



245

In [66]:
grouped.apply(lambda x: len(x)).value_counts()

1    245
dtype: int64

In [67]:
gb = full_labels.groupby('filename')
grouped_list = [gb.get_group(x) for x in gb.groups]
len(grouped_list)
train_index = np.random.choice(len(grouped_list), size=160, replace=False)
test_index = np.setdiff1d(list(range(200)), train_index)
len(train_index), len(test_index)
# take first 200 files
train = pd.concat([grouped_list[i] for i in train_index])
test = pd.concat([grouped_list[i] for i in test_index])
len(train), len(test)
train.to_csv('./data/train_labels.csv', index=None)
test.to_csv('./data/test_labels.csv', index=None)

 ## Generating test and train TFRecords

#Use generate_tfrecords.py to generate tfrecords for test and train.